In [ ]:
class VNA():
    import serial,sys,time,math,datetime,os, numpy as np, matplotlib.pyplot as plt
    from timeit import default_timer as timer
    
    def __init__():
        self.bla = 'bla'

    def setup_adapter(deviceport="/dev/ttyUSB2"):
        ser = serial.Serial()
        ser.port = deviceport
        ser.baudrate = 115200
        ser.timeout = 2.0
        ser.open()
        time.sleep(0.2)
        ser.flushInput()
        ser.write('++savecfg 0\n'.encode('ascii'))
        ser.write('++auto 0\n'.encode('ascii'))
        set_tm()

    def read_line_gpib():
        ser.write('++read eoi\n'.encode('ascii'))
        return ser.readline().decode().rstrip()

    def read_all_gpib():
        time.sleep(5)
        ser.write('++read eoi\n'.encode('ascii'))
        ser.timeout = 0.5    
        resp = bytes()
        cnt = 0
        lentot = 0
        for i in range(1,5000):
            resppart = ser.read(10000)
            time.sleep(0.01)
            #print(len(resppart))
            if (len(resppart) == 0):
                cnt += 1
            else:
                lentot += len(resppart)
                cnt = 0
                resp += resppart
            if (cnt>200):
                break 
        print('Read {} bytes'.format(lentot))
        return resp

    def read_all_gpib_withtarget(amt):
        time.sleep(5)    
        ser.timeout = 0.5   
        resp = bytes()
        cnt = 0
        lentot = 0
        st = timer()
        ser.write('++read eoi\n'.encode('ascii'))
        for i in range(1,5000):
            if (timer()-st > 3):
                ser.write('++read eoi\n'.encode('ascii'))
                st = timer()
            time.sleep(0.01)
            resppart = ser.read(100000)
            time.sleep(0.01)
            print(len(resppart))
            if (len(resppart) == 0):
                cnt += 1
            else:
                lentot += len(resppart)
                cnt = 0
                resp += resppart
            if (cnt>200):
                break 
            if (lentot>=amt):
                break
        print('Read {} bytes'.format(lentot))
        ser.timeout = 0.5
        return resp

    def wtf():   
        return ser.read(64000).decode('ascii').rstrip()

    def flush_gpib():    
        ser.write('++read eoi\n'.encode('ascii'))
        time.sleep(3.1)
        return ser.read(512000).decode('ascii').rstrip()

    def set_tm(tm=3000):
        ser.write('++read_tmo_ms {}\n'.format(tm).encode('ascii'))

    def send_command_with_resp(cmd):
        ser.write('{}\n'.format(cmd).encode('ascii'))
        time.sleep(0.05)
        return read_line_gpib()

    def send_command_no_resp(cmd):
        ser.write('{}\n'.format(cmd).encode('ascii'))

    def await_confirmation(tm):
        start = timer();
        while (read_line_gpib() != '1' and timer()-start < tm):
            time.sleep(1)
        print("Confirmation received in {:3.2f}s".format(timer()-start))

    def get_value_lists(text):
        ans3 = text.split('\n')
        ilist = []
        qlist = []
        for s in ans3:
            if s != '':
                #print(s)
                strarr = s.split(',')        
                ilist.append(float(strarr[0]))
                qlist.append(float(strarr[1]))
        return [ilist,qlist]

    def get_current_vna_state():
        flush_gpib()
        start = float(send_command_with_resp('STAR?;'))
        npoints = int(float(send_command_with_resp('POIN?;')))
        span = float(send_command_with_resp('SPAN?;'))
        ifbw = int(float(send_command_with_resp('IFBW?;')))
        #pwr = float(send_command_with_resp('POWE?;'))
        pwr = -10.0
        print("Current settings: [{} to {}] with {} points, IFBW of {} at {} dBm".format(start,start+span,npoints,ifbw,pwr))

    def get_new_log_name(logroot='/home/pi/Desktop/DataNX/vna/'):
        date = datetime.datetime.now()
        logdir = logroot+date.strftime('%Y%m%d')
        #if not os.path.isdir(logdir):
        #    print("Directory |{}| not there, making".format(logdir))
        #    os.mkdir(logdir)
        dirs = os.listdir(logroot)
        print(dirs)
        logfile = logdir+os.path.sep+date.strftime('vnasweep-%H-%M-%S.txt')
        print(logfile)

    def save_to_log(data):
        fl = open(logfile, "w")
        time.sleep(0.05)
        fl.write('DATE: {}\n'.format(date.strftime('%Y/%m/%d')))
        fl.write('TIME: {}\n'.format(date.strftime('%H:%M:%S')))
        fl.write('POWER: {}\n'.format(pwr))
        fl.write('IFBW: {}\n'.format(ifbw))
        fl.write('STIMULUS, REAL, IMAGINARY\n')
        fl.write('\n')
        freqs = list(np.linspace(start,start+span,npoints))
        [il,ql] = get_value_lists(response.decode('ascii'))
        if len(freqs) != len(il):
            print("BAD")
        else:
            for i in range(0,len(freqs)):
                try:
                    fl.write('{:12.10E}, {:12.10E}, {:12.10E}\n'.format(freqs[i],il[i],ql[i]))
                    print(i)
                except Exception as e:
                    print(e)def setup_adapter():
        ser = serial.Serial()
        ser.port = "/dev/ttyUSB2"
        ser.baudrate = 115200
        ser.timeout = 2.0
        ser.open()
        time.sleep(0.2)
        ser.flushInput()
        ser.write('++savecfg 0\n'.encode('ascii'))
        ser.write('++auto 0\n'.encode('ascii'))
        set_tm()

    def read_line_gpib():
        ser.write('++read eoi\n'.encode('ascii'))
        return ser.readline().decode().rstrip()

    def read_all_gpib():
        time.sleep(5)
        ser.write('++read eoi\n'.encode('ascii'))
        ser.timeout = 0.5    
        resp = bytes()
        cnt = 0
        lentot = 0
        for i in range(1,5000):
            resppart = ser.read(10000)
            time.sleep(0.01)
            #print(len(resppart))
            if (len(resppart) == 0):
                cnt += 1
            else:
                lentot += len(resppart)
                cnt = 0
                resp += resppart
            if (cnt>200):
                break 
        print('Read {} bytes'.format(lentot))
        return resp

    def read_all_gpib_withtarget(amt):
        time.sleep(5)    
        ser.timeout = 0.5   
        resp = bytes()
        cnt = 0
        lentot = 0
        st = timer()
        ser.write('++read eoi\n'.encode('ascii'))
        for i in range(1,5000):
            if (timer()-st > 3):
                ser.write('++read eoi\n'.encode('ascii'))
                st = timer()
            time.sleep(0.01)
            resppart = ser.read(100000)
            time.sleep(0.01)
            print(len(resppart))
            if (len(resppart) == 0):
                cnt += 1
            else:
                lentot += len(resppart)
                cnt = 0
                resp += resppart
            if (cnt>200):
                break 
            if (lentot>=amt):
                break
        print('Read {} bytes'.format(lentot))
        ser.timeout = 0.5
        return resp

    def wtf():   
        return ser.read(64000).decode('ascii').rstrip()

    def flush_gpib():    
        ser.write('++read eoi\n'.encode('ascii'))
        time.sleep(3.1)
        return ser.read(512000).decode('ascii').rstrip()

    def set_tm(tm=3000):
        ser.write('++read_tmo_ms {}\n'.format(tm).encode('ascii'))

    def send_command_with_resp(cmd):
        ser.write('{}\n'.format(cmd).encode('ascii'))
        time.sleep(0.05)
        return read_line_gpib()

    def send_command_no_resp(cmd):
        ser.write('{}\n'.format(cmd).encode('ascii'))

    def await_confirmation(tm):
        start = timer();
        while (read_line_gpib() != '1' and timer()-start < tm):
            time.sleep(1)
        print("Confirmation received in {:3.2f}s".format(timer()-start))

    def get_value_lists(text):
        ans3 = text.split('\n')
        ilist = []
        qlist = []
        for s in ans3:
            if s != '':
                #print(s)
                strarr = s.split(',')        
                ilist.append(float(strarr[0]))
                qlist.append(float(strarr[1]))
        return [ilist,qlist]

    def print_current_state():
        flush_gpib()
        start = float(send_command_with_resp('STAR?;'))
        npoints = int(float(send_command_with_resp('POIN?;')))
        span = float(send_command_with_resp('SPAN?;'))
        ifbw = int(float(send_command_with_resp('IFBW?;')))
        #pwr = float(send_command_with_resp('POWE?;'))
        pwr = -10.0
        print("Current settings: [{} to {}] with {} points, IFBW of {} at {} dBm".format(start,start+span,npoints,ifbw,pwr))

    def get_new_log_name():
        date = datetime.datetime.now()
        logroot = '/home/pi/Desktop/DataNX/vna/'
        logdir = logroot+date.strftime('%Y%m%d')
        #if not os.path.isdir(logdir):
        #    print("Directory |{}| not there, making".format(logdir))
        #    os.mkdir(logdir)
        dirs = os.listdir(logroot)
        print(dirs)
        logfile = logdir+os.path.sep+date.strftime('vnasweep-%H-%M-%S.txt')
        print(logfile)

    def save_to_log(data):
        fl = open(logfile, "w")
        time.sleep(0.05)
        fl.write('DATE: {}\n'.format(date.strftime('%Y/%m/%d')))
        fl.write('TIME: {}\n'.format(date.strftime('%H:%M:%S')))
        fl.write('POWER: {}\n'.format(pwr))
        fl.write('IFBW: {}\n'.format(ifbw))
        fl.write('STIMULUS, REAL, IMAGINARY\n')
        fl.write('\n')
        freqs = list(np.linspace(start,start+span,npoints))
        [il,ql] = get_value_lists(response.decode('ascii'))
        if len(freqs) != len(il):
            print("BAD")
        else:
            for i in range(0,len(freqs)):
                try:
                    fl.write('{:12.10E}, {:12.10E}, {:12.10E}\n'.format(freqs[i],il[i],ql[i]))
                    print(i)
                except Exception as e:
                    print(e)